In [155]:
from scipy import misc
from skimage.filters import threshold_otsu
import os
import numpy as np
from skimage import color
import numpy as np
from skimage import data
from matplotlib import pyplot as plt
lstrowtestCorrect=[]
#changing test set to binary images and summing the row and col to create histograms

for i in os.listdir("/Users/adamjankelow/documents/python/TEST/correct/"):
    if i == ".DS_Store" :
        print("ignore")
    else:
        k = misc.imread("/Users/adamjankelow/documents/python/TEST/correct/"+i)
        GrayScale = color.rgb2gray(k)
        thresh = threshold_otsu(GrayScale)
        GrayScale[GrayScale<thresh]=0
        GrayScale[GrayScale>thresh]=1
        # changing grayscale to binary
        Binary =GrayScale[0:2400, 0:2400] #cropping square images
        
        row = Binary.sum(axis=1, dtype=None, out=None, keepdims=False)
        lstrowtestCorrect.append(row) #lstrowtest is the row histograms of the test set's correct images

ignore


In [156]:
lstrowtestWrong=[]
for i in os.listdir("/Users/adamjankelow/documents/python/TEST/wrong/"):
    if i == ".DS_Store" :
        print("ignore")
    else:
        k = misc.imread("/Users/adamjankelow/documents/python/TEST/wrong/"+i)
        GrayScale = color.rgb2gray(k)
        thresh = threshold_otsu(GrayScale)
        GrayScale[GrayScale<thresh]=0
        GrayScale[GrayScale>thresh]=1
        # changing grayscale to binary
        Binary =GrayScale[0:2400, 0:2400] #cropping square images
        
        row = Binary.sum(axis=1, dtype=None, out=None, keepdims=False)
        lstrowtestWrong.append(row) #lstrowtest is the row histograms of the test set's wrong images

ignore


In [157]:
lstrowtrainCorrect=[]
#changing training set to binary images and summing the row and col to create histograms
for i in os.listdir("/Users/adamjankelow/documents/python/TRAIN/correct"):
    if i == ".DS_Store" :
        print("ignore")
    else:
        m = misc.imread("/Users/adamjankelow/documents/python/TRAIN/correct/"+i)
        n = color.rgb2gray(m)
        thresh = threshold_otsu(n)
        n[n<thresh]=0
        n[n>thresh]=1
        Binary2 =n[0:2400, 0:2400]#cropping square images
        # changing grayscale to binary
        
        row = Binary2.sum(axis=1, dtype=None, out=None, keepdims=False) 
        lstrowtrainCorrect.append(row) #lstrowtrain is the row histograms of the traning set



ignore


In [158]:
lstrowtrainWrong=[]
#changing training set to binary images and summing the row and col to create histograms
for i in os.listdir("/Users/adamjankelow/documents/python/TRAIN/wrong/"):
    if i == ".DS_Store" :
        print("ignore")
    else:
        m = misc.imread("/Users/adamjankelow/documents/python/TRAIN/wrong/"+i)
        n = color.rgb2gray(m)
        thresh = threshold_otsu(n)
        n[n<thresh]=0
        n[n>thresh]=1
        Binary2 =n[0:2400, 0:2400]#cropping square images
        # changing grayscale to binary
        
        row = Binary2.sum(axis=1, dtype=None, out=None, keepdims=False) 
        lstrowtrainWrong.append(row) #lstrowtrain is the row histograms of the training set



ignore


In [159]:
#PCA reduction ( 20 dimension gives most accurate results for this data set)

In [242]:
#PCAtrainCorrect is the traning set that has had PCA reduction and has the correctimages
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pcatr = sklearnPCA(n_components=(6))
PCAtrainCorrect = sklearn_pcatr.fit_transform(lstrowtrainCorrect)

sklearn_pcatrw = sklearnPCA(n_components=(6))
PCAtrainWrong = sklearn_pcatrw.fit_transform(lstrowtrainWrong)

In [243]:

sklearn_pcate = sklearnPCA(n_components=6)
PCAtestCorrect = sklearn_pcate.fit_transform(lstrowtestCorrect)

sklearn_pcatew = sklearnPCA(n_components=6)
PCAtestWrong = sklearn_pcatew.fit_transform(lstrowtestWrong)

In [244]:
print(PCAtrainCorrect[1])

[ 1451.50076696  1734.89559994 -1793.64295986  -330.07306974   753.23865275
  -201.09178581]


In [245]:
# changing training set from a numpy array to normal list

TrainingSetCorrect=[]
for RowSum in PCAtrainCorrect:
    RowSumlstCorrect = RowSum.tolist()
  
    TrainingSetCorrect.append(RowSumlstCorrect)
    

TrainingSetWrong=[]
for RowSum in PCAtrainWrong:
    RowSumlstWrong = RowSum.tolist()
  
    TrainingSetWrong.append(RowSumlstWrong)
    
       

In [246]:
#changing test set from numpy array to list 
TestSetCorrect=[]
 
for Rowsum in PCAtestCorrect: 
    Rowsumlst = Rowsum.tolist()

    TestSetCorrect.append(Rowsumlst)

TestSetWrong=[]
 
for Rowsum1 in PCAtestWrong: 
    Rowsumlst1 = Rowsum1.tolist()

    TestSetWrong.append(Rowsumlst1)




In [247]:
for image in TrainingSetCorrect:
    image.append("yes")

for img in TrainingSetWrong:
    img.append("no")




In [248]:
# get neighbours returns the k nearest neighbors of the image in the test set
import operator 
def getNeighbors(trainingSet, testInstance, k):
	distances = []
	length = len(testInstance)-1
	for x in range(len(trainingSet)):
		dist = euclideanDistance(testInstance, trainingSet[x], length)
		distances.append((trainingSet[x], dist))
	distances.sort(key=operator.itemgetter(1))
	neighbors = []
	for x in range(k):
		neighbors.append(distances[x][0])
	return neighbors

# function used in k nearest neighbor
import math
def euclideanDistance(instance1, instance2, length):
	distance = 0
	for x in range(length):
		distance += pow((instance1[x] - instance2[x]), 2)
	return math.sqrt(distance)

In [249]:
# using k nearest neighbors to predict response of test data


PrdResponselstCorrect=[]
for image in TestSetCorrect:
    yes=0
    no=0
    trainSet = TrainingSetCorrect+TrainingSetWrong
    testInstance = image
    k = 3
    neighbors = getNeighbors(trainSet, testInstance, k)
    
    for neighbor in neighbors:
        if (neighbor[-1])== "yes":
            yes+=1
        else:
            no+=1
            
    if yes> no:
        PrdResponselstCorrect.append("yes")
    else:
        PrdResponselstCorrect.append("no")

PrdResponselstWrong=[]
for image in TestSetWrong:
    yes=0
    no=0
    trainSet = TrainingSetCorrect+TrainingSetWrong
    testInstance = image
    k = 3
    neighbors = getNeighbors(trainSet, testInstance, k)
    
    for neighbor in neighbors:
        if (neighbor[-1])== "yes":
            yes+=1
        else:
            no+=1
            
    if yes> no:
        PrdResponselstWrong.append("yes")
    else:
        PrdResponselstWrong.append("no")      


print(PrdResponselstCorrect)

print()
print(PrdResponselstWrong)




['no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'yes', 'no', 'yes', 'no', 'yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes']
()
['yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no']


In [250]:
#CorrectResponseis  list of what the actual responses should be for the correct images

CorrectResponse =[]
WrongResponse=[]

for i in TestSetCorrect:
    CorrectResponse.append("yes")

for i in TestSetWrong:
    WrongResponse.append("no")
    
     

In [252]:
# accuracy test for predicted results (84% is currently the highest)
count=0
Correct=0
Wrong=0

for i in CorrectResponse:
   
    if CorrectResponse[count] is PrdResponselstCorrect[count]:
        Correct+=1
    else:
        Wrong+=1
    count+=1
# accuracy test for predicted results (84% is currently the highest)
cnt=0

for i in WrongResponse:
    if WrongResponse[cnt] is PrdResponselstWrong[cnt]:
        Correct+=1
    else:
        Wrong+=1
    cnt+=1
    
test=TestSetCorrect+TestSetWrong
size=float(len(test))
print((Correct/size)*100)

    





64.6341463415


47


82.0


int

float